<div style="text-align: center;">
  <font size="5">乳がんの組織検査の自動判定器</font>
</div>
<div style="text-align: right;">
  <font size="3">cc by Shigeto R. Nishitani</font>
</div>

* 2017-11-17 関西学院大学，理工学部，数値計算
* /Users/bob/Github/TeamNishitani/coding_the_matrix/c8_breast_cancer_maple.ipynb



# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Breast-Cancer-Wisconsin-(Diagnostic)-Data-Set" data-toc-modified-id="Breast-Cancer-Wisconsin-(Diagnostic)-Data-Set-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Breast Cancer Wisconsin (Diagnostic) Data Set</a></div><div class="lev2 toc-item"><a href="#Attribute-Information:" data-toc-modified-id="Attribute-Information:-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Attribute Information:</a></div><div class="lev2 toc-item"><a href="#分類器" data-toc-modified-id="分類器-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>分類器</a></div><div class="lev2 toc-item"><a href="#仮説クラス" data-toc-modified-id="仮説クラス-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>仮説クラス</a></div><div class="lev1 toc-item"><a href="#最急降下法" data-toc-modified-id="最急降下法-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>最急降下法</a></div><div class="lev1 toc-item"><a href="#maple-code" data-toc-modified-id="maple-code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>maple code</a></div><div class="lev2 toc-item"><a href="#show_vector" data-toc-modified-id="show_vector-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>show_vector</a></div><div class="lev2 toc-item"><a href="#データ読み込み" data-toc-modified-id="データ読み込み-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>データ読み込み</a></div><div class="lev2 toc-item"><a href="#wの初期化と最小化" data-toc-modified-id="wの初期化と最小化-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>wの初期化と最小化</a></div><div class="lev1 toc-item"><a href="#結果" data-toc-modified-id="結果-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>結果</a></div><div class="lev2 toc-item"><a href="#評価(1)" data-toc-modified-id="評価(1)-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>評価(1)</a></div><div class="lev2 toc-item"><a href="#評価(2)-validate.data" data-toc-modified-id="評価(2)-validate.data-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>評価(2) validate.data</a></div>

# Breast Cancer Wisconsin (Diagnostic) Data Set

<https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)>

## Attribute Information:

1. ID number 
1. Diagnosis (M = malignant, B = benign) M:悪性，B:良性
1. 3-32

Ten real-valued features are computed for each cell nucleus: 

* 半径radius (mean of distances from center to points on the perimeter) 
* テクスチャtexture (standard deviation of gray-scale values) 
* 境界の長さperimeter 
* 面積area 
* なめらかさsmoothness (local variation in radius lengths) 
* コンパクトさcompactness (perimeter^2 / area - 1.0) 
* くぼみ度合いconcavity (severity of concave portions of the contour) 
* くぼみの数concave points (number of concave portions of the contour) 
* 対称性symmetry 
* フラクタル次元fractal dimension ("coastline approximation" - 1)

http://people.idsia.ch/~juergen/deeplearningwinsMICCAIgrandchallenge.html

## 分類器
与えられた特徴ベクトル$\boldsymbol{a}$に対し，
細胞組織が悪性か良性かを分類する関数$C(\boldsymbol{y})$を選び出すプログラムを作成しよう．

## 仮説クラス
分類器は可能な分類器の集合(**仮説クラス**)から選ばれる．この場合，仮説クラスとは特徴ベクトルの空間$\mathbb{R}^D$から$\mathbb{R}$への線形関数$h(\cdot)$である．すると分類器は次のような関数として定義される．

$$
C(\boldsymbol{y}) = 
\left\{ \begin{array}{ccc}
+1 &  {\rm when} & h(\boldsymbol{y})\geq 0\\
-1 &  {\rm when} & h(\boldsymbol{y})<0
\end{array} \right.
$$

各線形関数$h:\mathbb{R}^D \rightarrow \mathbb{R}$に対して，
次のような$D$ベクトル$\boldsymbol{w}$が存在する．
$$
h(\boldsymbol{y}) = \boldsymbol{w}\cdot \boldsymbol{y}
$$
したがって，そのような線形関数を選ぶことは，結局$D$ベクトル$\boldsymbol{w}$を
選ぶことに等しい．特に，$\boldsymbol{w}$を選ぶことは，仮説クラス$h$を
選ぶことと等価なので，$\boldsymbol{w}$を**仮説ベクトル**と呼ぶ．

単に，ベクトルの掛け算で分類器はできそう．問題はどうやってこの仮説ベクトルを決定するか？ですよね．


# 最急降下法

損失関数に
$$
L(w)=\sum_{i=1}^m (a_i \cdot w - b_i)^2
$$
を選ぶと
$$
\begin{aligned}
\frac{\partial L}{\partial w_j} &= 
\sum_{i=1}^m \frac{\partial}{\partial w_j}(a_i \cdot w -b_i)^2 \\
&= \sum_{i=1}^m 2(a_i \cdot w -b_i) a_{ij}
\end{aligned}
$$
となる．
ここで，$a_{ij}$は$a_i$の$j$番目の要素です．
こいつを勾配として，local minimumを求める．

このsumはiのmまでの集計と記述していますが，テキストではデータ数の和を意図しています．jはベクトル$w$の要素となります．

# maple code

* file:/Users/bob/Github/TeamNishitani/coding_the_matrix/codes/my_cancer_detector.mw

## show_vector
``` octave
restart;
with(LinearAlgebra):
show_vector:=proc(v)
  local params,i,j;s
  params := ["radius", "texture", "perimeter","area",
    "smoothness","compactness","concavity",
    "concave points","symmetry","fractal dimension"];
  printf("     (params)     : ");
  printf("     (mean)    (stderr)     (worst)");
  for j from 1 to 10 do
    printf("\n%17s :",params[j]);
    for i from 1 to 3 do 
      printf("%12.8f", v[j*3+i-3]); 
    end;
  end;
end:
```

## データ読み込み

``` octave
A := ImportMatrix("./train_A.data"):
b := ImportVector("./train_b.data"):
```

## wの初期化と最小化
``` octave
w := Vector(30,0.0001):
res:=A.w-b;
sigma:=2.0*10^(-9);
for i from 1 to 3000 do
  res:=A.w-b;
  dLw:=sigma*(res.A);
  w:=w-Transpose(dLw);
end:
```

``` octave
> show_vector(w);
     (params)     :      (mean)    (stderr)     (worst)
           radius :  0.00143827  0.00207065  0.00759660
          texture :  0.00025364  0.00011450  0.00008829
        perimeter :  0.00007122  0.00008563  0.00012780
             area :  0.00011216  0.00011144  0.00035082
       smoothness :  0.00007817 -0.00095866  0.00010154
      compactness :  0.00009903  0.00010026  0.00010009
        concavity :  0.00010414  0.00010054  0.00131594
   concave points :  0.00222308  0.00631956 -0.00184204
         symmetry :  0.00011618  0.00004085  0.00001965
fractal dimension :  0.00007506  0.00012514  0.00010802
```

# 結果

## 評価(1)

``` octave
show_correct_error:=proc(A, b, w)
# Diagnosis (M = malignant, B = benign) M:悪性(-1)，B:良性(1)
local correct, safe_error, critical_error,
      predict, bias, n, i;
correct,safe_error,critical_error:=0,0,0;
predict := A.w:
bias := 0.0;
n := Dimension(b);
for i from 1 to n do
  if (predict[i]-bias)*b[i]>0 then
    correct := correct+1;
  elif (predict[i]-bias)<0 and b[i]>0 then
    safe_error:= safe_error+1;
  elif (predict[i]-bias)>0 and b[i]<0 then
    critical_error:= critical_error+1;
  end;
end;
printf("       correct: %4d/%4d\n",correct,n);
printf("    safe error: %4d\n",safe_error);
printf("critical error: %4d",critical_error);
end:
```

```octave
> show_correct_error(A, b, w);
       correct:  261/ 300
    safe error:    0
critical error:   39
```

## 評価(2) validate.data

``` octave
> A := ImportMatrix("./validate_A.data"):
> b := ImportVector("./validate_b.data"):
> show_correct_error(A, b, w);
       correct:  247/ 260
    safe error:    1
critical error:   12
```